In [71]:

import tensorflow as tf
from keras.models import *
from keras.layers import *
from keras.applications.vgg16 import VGG16
from keras.regularizers import l2


from keras.applications.imagenet_utils import _obtain_input_shape
import keras.backend as K
from keras.optimizers import Adam, SGD

# from keras_contrib.applications import densenet

In [105]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import re

In [13]:
base_model = VGG16(include_top=False, input_tensor=Input(shape=(160, 576,3)))

In [62]:
def fcn_vgg_8(input_shape, num_classes, weight_decay=5e-4):
    img_input = Input(shape=input_shape)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    vgg_conv3_out = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(vgg_conv3_out)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    vgg_conv4_out = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(vgg_conv4_out)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    vgg_conv5_out = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
    
    # convert dense layers to fully conv layers
    x = Conv2D(4096, (7, 7), activation='relu', padding='same', name='fc1', kernel_regularizer=l2(weight_decay))(vgg_conv5_out)
    x = Dropout(0.5)(x)
    x = Conv2D(4096, (1, 1), activation='relu', padding='same', name='fc2', kernel_regularizer=l2(weight_decay))(x)
    x = Dropout(0.5)(x)
    conv7_out = Conv2D(num_classes, (1, 1), activation='linear', padding='valid', strides=(1, 1), kernel_regularizer=l2(weight_decay), name='decoder_conv7')(x)   
    
    # decoder network
    vgg_conv4_conv1x1 = Conv2D(num_classes, (1, 1), activation='relu', padding='valid', strides=(1, 1), kernel_regularizer=l2(weight_decay),
                              name="layer4_conv1x1")(vgg_conv4_out)
    
    vgg_conv3_conv1x1 = Conv2D(num_classes, (1, 1), activation='relu', padding='valid', strides=(1, 1), kernel_regularizer=l2(weight_decay),
                              name="layer3_conv1x1")(vgg_conv3_out)
    
    upsample1 = Conv2DTranspose(num_classes, kernel_size=4, strides=(2,2), padding='same', kernel_regularizer=l2(weight_decay), name='upsample_1')(conv7_out)
    
    skip1 = Add()([upsample1, vgg_conv4_conv1x1])
    upsample2 = Conv2DTranspose(num_classes, kernel_size=4, strides=(2,2), padding='same', kernel_regularizer=l2(weight_decay), name='upsample_2')(skip1)
    
    skip2 = Add()([upsample2, vgg_conv3_conv1x1])
    upsample3 = Conv2DTranspose(num_classes, kernel_size=16, strides=(8,8), padding='same', kernel_regularizer=l2(weight_decay), name='upsample_3')(skip2)
    
#     model = Model(img_input, upsample1)
    
    model = Model(img_input, upsample3)

    weights_path = os.path.expanduser(os.path.join('~', '.keras/models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'))
    model.load_weights(weights_path, by_name=True)
    return model

In [63]:
model_fcn = fcn_vgg_8((160, 576, 3), 2, weight_decay=5e-4)
model_fcn.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_24 (InputLayer)           (None, 160, 576, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 160, 576, 64) 1792        input_24[0][0]                   
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 160, 576, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 80, 288, 64)  0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [68]:
# freeze layers from vgg for now
for idx, layer in enumerate(model_fcn.layers):
    if idx < 17:
        layer.trainable = False

In [69]:
model_fcn.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_24 (InputLayer)           (None, 160, 576, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 160, 576, 64) 1792        input_24[0][0]                   
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 160, 576, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 80, 288, 64)  0           block1_conv2[0][0]               
__________________________________________________________________________________________________
block2_con

In [78]:
def fcn_binary_crossentropy(y_true, y_pred, from_logits=False):
    pred_shape = K.int_shape(y_pred)
    true_shape = K.int_shape(y_true)
    y_pred_reshaped = K.reshape(y_pred, (-1, 2))
    y_true_reshaped = K.reshape(y_true, (-1, 2))

    result = K.binary_crossentropy(y_pred_reshaped, y_true_reshaped,
                                   from_logits=from_logits)
    result = K.mean(result, axis=-1)

    if len(true_shape) >= 3:
        return K.reshape(result, true_shape[:-1])
    else:
        return result
    
def fcn_binary_crossentropy_with_regularisation(y_true, y_pred, from_logits=False):
    loss1 = fcn_binary_crossentropy(y_true, y_pred, from_logits=from_logits)
    loss2 = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES) + 5e-4 + K.sum(tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES))
    
    return K.sum(loss1, loss2)




In [87]:
# https://raw.githubusercontent.com/theduynguyen/Keras-FCN/master/loss_func.py

def fcn_xent(y_true, y_pred):
	y_true_reshaped = K.flatten(y_true)
	y_pred_reshaped = K.flatten(y_pred)

	return K.binary_crossentropy(y_pred_reshaped, y_true_reshaped)


def pixel_acc(y_true, y_pred):
	s = K.shape(y_true)

	# reshape such that w and h dim are multiplied together
	y_true_reshaped = K.reshape( y_true, tf.stack( [-1, s[1]*s[2], s[-1]] ) )
	y_pred_reshaped = K.reshape( y_pred, tf.stack( [-1, s[1]*s[2], s[-1]] ) )

	# correctly classified
	clf_pred = K.one_hot( K.argmax(y_pred_reshaped),s[-1])
	correct_pixels_per_class = K.cast( K.equal(clf_pred,y_true_reshaped), dtype='float32')

	return K.sum(correct_pixels_per_class) / K.cast(K.prod(s), dtype='float32')

def mean_acc(y_true, y_pred):
	s = K.shape(y_true)

	# reshape such that w and h dim are multiplied together
	y_true_reshaped = K.reshape( y_true, tf.stack( [-1, s[1]*s[2], s[-1]] ) )
	y_pred_reshaped = K.reshape( y_pred, tf.stack( [-1, s[1]*s[2], s[-1]] ) )

	# correctly classified
	clf_pred = K.one_hot( K.argmax(y_pred_reshaped), nb_classes = s[-1])
	equal_entries = K.cast(K.equal(clf_pred,y_true_reshaped), dtype='float32') * y_true_reshaped

	correct_pixels_per_class = K.sum(equal_entries, axis=1)
	n_pixels_per_class = K.sum(y_true_reshaped,axis=1)

	acc = correct_pixels_per_class / n_pixels_per_class
	acc_mask = tf.is_finite(acc)
	acc_masked = tf.boolean_mask(acc,acc_mask)

	return K.mean(acc_masked)

def mean_IoU(y_true, y_pred):
	s = K.shape(y_true)

	# reshape such that w and h dim are multiplied together
	y_true_reshaped = K.reshape( y_true, tf.stack( [-1, s[1]*s[2], s[-1]] ) )
	y_pred_reshaped = K.reshape( y_pred, tf.stack( [-1, s[1]*s[2], s[-1]] ) )

	# correctly classified
	clf_pred = K.one_hot( K.argmax(y_pred_reshaped), nb_classes = s[-1])
	equal_entries = K.cast(K.equal(clf_pred,y_true_reshaped), dtype='float32') * y_true_reshaped

	intersection = K.sum(equal_entries, axis=1)
	union_per_class = K.sum(y_true_reshaped,axis=1) + K.sum(y_pred_reshaped,axis=1)

	iou = intersection / (union_per_class - intersection)
	iou_mask = tf.is_finite(iou)
	iou_masked = tf.boolean_mask(iou,iou_mask)

	return K.mean( iou_masked )

In [79]:
lr_base = 0.01
fcn_optimizer = SGD(lr=lr_base, momentum=0.9)
# https://stackoverflow.com/questions/49715192/tensorflow-mean-iou-for-just-foreground-class-for-binary-semantic-segmentation/50266195#50266195

def keras_mean_iou(labels, predictions):
    """
    labels,prediction with shape of [batch,height,width,class_number=2]
    """
    mean_iou = K.variable(0.0)
    seen_classes = K.variable(0.0)

    for c in range(2):
        labels_c = K.cast(K.equal(labels, c), K.floatx())
        pred_c = K.cast(K.equal(predictions, c), K.floatx())

        labels_c_sum = K.sum(labels_c)
        pred_c_sum = K.sum(pred_c)

        intersect = K.sum(labels_c*pred_c)
        union = labels_c_sum + pred_c_sum - intersect
        iou = intersect / union
        condition = K.equal(union, 0)
        mean_iou = K.switch(condition,
                            mean_iou,
                            mean_iou+iou)
        seen_classes = K.switch(condition,
                                seen_classes,
                                seen_classes+1)

    mean_iou = K.switch(K.equal(seen_classes, 0),
                        mean_iou,
                        mean_iou/seen_classes)
    return mean_iou

In [88]:
# model_fcn.compile(loss=fcn_binary_crossentropy_with_regularisation,
#                   optimizer=Adam(),
#                   metrics=[keras_mean_iou])

model_fcn.compile(loss=fcn_xent,
                  optimizer=Adam(),
                  metrics=[mean_iou, pixel_acc])

In [100]:
np.random.seed(10)
road_imgs_gt = glob.glob(os.path.join('./data', 'data_road/training', 'gt_image_2', '*_road_*.png'))
np.random.shuffle(road_imgs_gt)

In [101]:
road_imgs_gt[:10]

['./data/data_road/training/gt_image_2/uu_road_000013.png',
 './data/data_road/training/gt_image_2/um_road_000061.png',
 './data/data_road/training/gt_image_2/umm_road_000092.png',
 './data/data_road/training/gt_image_2/um_road_000043.png',
 './data/data_road/training/gt_image_2/umm_road_000075.png',
 './data/data_road/training/gt_image_2/uu_road_000016.png',
 './data/data_road/training/gt_image_2/umm_road_000078.png',
 './data/data_road/training/gt_image_2/umm_road_000036.png',
 './data/data_road/training/gt_image_2/umm_road_000031.png',
 './data/data_road/training/gt_image_2/umm_road_000062.png']

In [103]:
val_gt = road_imgs_gt[:10]
val_img = [x.replace('gt_image_2', 'image_2') for x in val_gt]
train_gt = road_imgs_gt[10:]
train_img = [x.replace('gt_image_2', 'image_2') for x in train_gt]

In [ ]:
def train_generator(train_files_list, val_files_list):
    """
    Create batches of training data
    :param batch_size: Batch Size
    :return: Batches of training data
    """
    image_paths = train_img
    label_paths = {
        re.sub(r'_(lane|road)_', '_', os.path.basename(path)): path
        for path in train_gt}
    background_color = np.array([255, 0, 0])

    while True:
        images = []
        gt_images = []
        for image_file in image_paths[batch_i:batch_i+batch_size]:
            gt_image_file = label_paths[os.path.basename(image_file)]

            image = scipy.misc.imresize(scipy.misc.imread(image_file), image_shape)
            gt_image = scipy.misc.imresize(scipy.misc.imread(gt_image_file), image_shape)

            gt_bg = np.all(gt_image == background_color, axis=2)
            gt_bg = gt_bg.reshape(*gt_bg.shape, 1)
            gt_image = np.concatenate((gt_bg, np.invert(gt_bg)), axis=2)

            images.append(image)
            gt_images.append(gt_image)

        yield np.array(images), np.array(gt_images)

In [106]:
{re.sub(r'_(lane|road)_', '_', os.path.basename(path)): path for path in val_gt}

{'um_000043.png': './data/data_road/training/gt_image_2/um_road_000043.png',
 'um_000061.png': './data/data_road/training/gt_image_2/um_road_000061.png',
 'umm_000031.png': './data/data_road/training/gt_image_2/umm_road_000031.png',
 'umm_000036.png': './data/data_road/training/gt_image_2/umm_road_000036.png',
 'umm_000062.png': './data/data_road/training/gt_image_2/umm_road_000062.png',
 'umm_000075.png': './data/data_road/training/gt_image_2/umm_road_000075.png',
 'umm_000078.png': './data/data_road/training/gt_image_2/umm_road_000078.png',
 'umm_000092.png': './data/data_road/training/gt_image_2/umm_road_000092.png',
 'uu_000013.png': './data/data_road/training/gt_image_2/uu_road_000013.png',
 'uu_000016.png': './data/data_road/training/gt_image_2/uu_road_000016.png'}